In [1]:
#include <ranges>
#include <iostream>
#include <vector>
#include "chapter07/sum.hpp"
#include <algorithm>
#include <utility>
#include <numeric>
#include <string_view>
#include <list>
#include <map>
#include <concepts>

In [2]:
constexpr auto res = std::views::iota(1) |
    std::views::transform([](auto n){return n*n; }) | 
    std::views::filter([](auto n){return n % 2 == 1;}) |
    std::views::take_while([](auto n) {return n < 10000;});

In [3]:
res

(std::ranges::take_while_view<views::all_t<filter_view<transform_view<iota_view<int, unreachable_sentinel_t>, (lambda)>, (lambda)> >, (lambda)> &) @0x7fd19554d000


In [4]:
std::vector v1{1,2,3,4,5};

generic lambda can be included in a header file for cling for correct compilation.
The cause can be found here: https://github.com/jupyter-xeus/xeus-cling/issues/258.

In [5]:
sum(v1);

(int) 15


In [6]:
int s0 = 0;
for(auto i : std::views::iota(1) |
    std::views::transform([](auto n){return n*n; }) | 
    std::views::filter([](auto n){return n % 2 == 1;}) |
    std::views::take_while([](auto n) {return n < 10000;})){
    s0 += i;
}

In [7]:
s0

(int) 166650


## 7.1 range access

In [8]:
namespace ns{
    struct Foo{};
    void swap(Foo&, Foo&) noexcept {std::cout << "custom swap" << std::endl;}
}

In [9]:
ns::Foo a, b;

In [10]:
std::swap(a, b);

(void) @0x7fd181ffa030


In [11]:
using std::swap;
swap(a, b);

custom swap


(void) @0x7fd181ffa030


In [12]:
std::ranges::swap(a, b);

custom swap


(void) @0x7fd181ffa030


In [13]:
{
    using std::ranges::swap;
    swap(a, b);
}

custom swap


In [14]:
auto const ints = {0, 1, 2, 3, 4, 5};

In [15]:
ints

(std::initializer_list<int> &) { 0, 1, 2, 3, 4, 5 }


In [16]:
auto even = [](int i) { return 0 == i % 2; };
auto square = [](int i) { return i * i; };

// the "pipe" syntax of composing the views:
for (int i : ints | std::views::filter(even) | std::views::transform(square))
    std::cout << i << ' ';

std::cout << '\n';

0 4 16 


(std::basic_ostream<char, std::char_traits<char> > &) @0x7fd195817500


In [17]:
for (int i : std::views::transform(std::views::filter(ints, even), square))
    std::cout << i << ' ';

0 4 16 

In [18]:
for (int i : std::views::iota(0, 6)
           | std::views::filter(even)
           | std::views::transform(square))
    std::cout << i << ' ';
std::cout << '\n';

0 4 16 


(std::basic_ostream<char, std::char_traits<char> > &) @0x7fd195817500


## 7.2 range concepts

In [19]:
template <typename R> 
concept is_range = requires(R& r){
    std::ranges::begin(r);
    std::ranges::end(r);
};

In [20]:
static_assert(is_range<std::vector<int>>);

In [21]:
static_assert(is_range<std::string>);

In [22]:
static_assert(is_range<int[10]>);

In [23]:
template<std::ranges::borrowed_range R>
void f(R&& r){

};

borrowed_range

In [24]:
// not compile
// f(std::vector<int>{1,2,3,4});

In [25]:
std::vector v_test{1,2,3,4};
f(v_test);

(void) @0x7fd181ffa030


In [26]:
f(std::string_view{"1234"});

(void) nullptr


sized_range

In [27]:
template<typename R>
concept is_sized_range = is_range<R> && requires(R &r) {std::ranges::size(r);};

In [28]:
is_sized_range<int[10]>

(bool) true


In [29]:
is_sized_range<std::array<int, 10>>

(bool) true


In [30]:
is_sized_range<std::list<int>>

(bool) true


view

In [31]:
std::ranges::view<std::string_view>

(bool) true


In [32]:
std::ranges::view<std::vector<int>>

(bool) false


In [33]:
std::ranges::view<std::string>

(bool) false


In [34]:
template<typename R>
concept is_view = is_range<R> && std::movable<R> && std::default_initializable<R> && std::ranges::enable_view<R>;

In [35]:
is_view<std::string>

(bool) false


In [36]:
is_view<std::vector<int>>

(bool) false


In [37]:
is_view<std::string_view>

(bool) true


In [38]:
std::ranges::view<std::span<int>>

(bool) true


other concepts

In [39]:
template<typename R>
concept is_common_range = is_range<R> && std::same_as<std::ranges::iterator_t<R>, std::ranges::sentinel_t<R>>;

In [40]:
is_common_range<std::vector<int>>

(bool) true


## 7.3 range components

view_interface example from cppreference website:

In [41]:
template<class T, class A>
class VectorView : public std::ranges::view_interface<VectorView<T, A>>
{
public:
    VectorView() = default;
 
    VectorView(const std::vector<T, A>& vec) :
        m_begin(vec.cbegin()), m_end(vec.cend())
    {}
 
    auto begin() const { return m_begin; }
 
    auto end() const { return m_end; }
 
private:
    typename std::vector<T, A>::const_iterator m_begin{}, m_end{};
};

In [42]:
{
    std::vector<int> v = {1, 4, 9, 16};
 
    VectorView view_over_v{v};
 
    // We can iterate with begin() and end().
    for (int n : view_over_v)
        std::cout << n << ' ';
    std::cout << '\n';
 
    // We get operator[] for free when inheriting from view_interface
    // since we satisfy the random_access_range concept.
    for (std::ptrdiff_t i = 0; i != view_over_v.size(); ++i)
        std::cout << "v[" << i << "] = " << view_over_v[i] << '\n';
}

1 4 9 16 
v[0] = 1
v[1] = 4
v[2] = 9
v[3] = 16


## 7.4 range factory

In [43]:
auto ev = std::ranges::empty_view<int>{};

In [44]:
ev.empty()

(bool) true


In [45]:
std::ranges::single_view single_view{6};

In [46]:
single_view.begin()

(int *) 0x7fd190111000


In [47]:
*single_view.begin()

(int) 6


In [48]:
single_view.end()

(int *) 0x7fd190111004


In [49]:
single_view.begin() + 1

(int *) 0x7fd190111004


In [50]:
for(auto e : std::ranges::iota_view(0, 5)){
    std::cout << e << ' ';
}
std::cout << std::endl;

0 1 2 3 4 


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fd195817500


## 7.5 range adapter

In [51]:
ints

(std::initializer_list<int> &) { 0, 1, 2, 3, 4, 5 }


In [52]:
auto even_view = ints | std::ranges::views::filter(even);

In [53]:
even_view

(std::ranges::filter_view<views::all_t<const initializer_list<int> &>, (lambda)> &) @0x7fd1900f0000


In [54]:
for(auto i : ints | std::ranges::views::take(2) | std::ranges::views::filter(even)){
    std::cout << i << ' ' ;
}

0 

In [55]:
for(auto i : ints | std::ranges::views::take_while([](auto n){return n <= 3;}) | std::ranges::views::filter(even)){
    std::cout << i << ' ' ;
}

0 2 

In [56]:
for(auto i : ints | std::ranges::views::drop(2) | std::ranges::views::filter(even)){
    std::cout << i << ' ' ;
}

2 4 

In [57]:
for(auto i : ints | std::ranges::views::drop_while([](auto n){ return n <= 3;}) | std::ranges::views::filter(even)){
    std::cout << i << ' ' ;
}

4 

In [58]:
std::vector<std::string_view> v_sv{"hello", " ", "world", "!"};

In [59]:
for(auto ch : v_sv | std::ranges::views::join){
    std::cout<< ch ;
}

hello world!

In [60]:
std::string_view test_sv{"a b c d e f g"};

In [61]:
for(auto word : test_sv | std::ranges::views::split(' ')){
    for(auto ch : word){
        std::cout << ch;
    }
    std::cout << std::endl;
}

a
b
c
d
e
f
g


In [62]:
for(auto i : ints | std::ranges::views::reverse){
    std::cout << i << std::endl;
}

5
4
3
2
1
0


In [63]:
std::map<std::string, double> prices{
    {"apple", 1.0},
    {"banana", 0.5},
}

In [64]:
for(const auto& key_viewed : std::ranges::views::keys(prices)){
    std::cout << key_viewed << std::endl;
}

apple
banana


elements

In [65]:
for(auto && name : prices | std::ranges::views::elements<0>){
    std::cout << name << std::endl;
}

apple
banana


In [66]:
for(auto && price : prices | std::ranges::views::elements<1>){
    std::cout << price << std::endl;
}

1
0.5


## 7.6 others

In [67]:
template<std::ranges::range R>
using iter_concept = typename std::ranges::iterator_t<R>;

In [68]:
std::same_as<iter_concept<std::vector<int>>, std::contiguous_iterator_tag>

(bool) false


In [69]:
std::same_as<iter_concept<std::vector<int>>::iterator_concept, std::contiguous_iterator_tag>

(bool) true


In [70]:
auto it = std::ranges::find(ints, 3)

In [71]:
it

(borrowed_iterator_t<const std::initializer_list<int> &>) @0x7fd181ffa030


In [72]:
*it

(const int) 3


In [73]:
std::vector<int> generate_vector(){
    return {1,2,3,4};
}

In [74]:
auto it_dangling = std::ranges::find(generate_vector(), 3);

In [75]:
it_dangling

(std::ranges::borrowed_iterator_t<std::vector<int, std::allocator<int> > > &) @0x7fd1900ad000


In [76]:
// compile error: indirection requires pointer operand
// *it_dangling

In [77]:
struct Point{
    int x;
    int y;
    int GetDistance() const{
        return std::abs(x) + std::abs(y);
    }
};

In [78]:
std::vector<Point> points{
    {1,2},
    {2,3},
    {0,0},
    {1,1}
}

In [79]:
std::ranges::sort(points, std::ranges::less{}, &Point::x);

(__gnu_cxx::borrowed_iterator_t<vector<Point, allocator<Point> > &>) @0x7fd16389da40


In [80]:
for(auto && p: points){
    std::cout << p.x << ' ' << p.y << std::endl; 
}

0 0
1 2
1 1
2 3


In [81]:
auto it_distance = std::ranges::find(points, 5, &Point::GetDistance)

In [82]:
it_distance->x

(int) 2


In [83]:
it_distance->y

(int) 3


## 7.7 Applications